In [1]:
import geopandas as gpd
import pandas as pd

from prompt2map.providers.openai import OpenAIProvider
from prompt2map.providers.openai import generate_openai_embedding_request
import topojson as tp

import logging

logging.basicConfig(level=logging.INFO)

# Carta Administrativa Oficial de Portugal

Fonte: https://snig.dgterritorio.gov.pt/rndg/srv/por/catalog.search#/metadata/198497815bf647ecaa990c34c42e932e

In [ ]:
!wget https://geo2.dgterritorio.gov.pt/caop/CAOP_Continente_2023-gpkg.zip -P raw

In [2]:
caop2023 = gpd.read_file('raw/CAOP_Continente_2023-gpkg.zip', layer="Cont_Freg_CAOP2023")

codigos_municipios = caop2023.DICOFRE.str[:4].drop_duplicates()
codigos_df = (caop2023[["DICOFRE", "Freguesia", "Municipio", "Distrito"]]
                .rename(columns={"DICOFRE": "DTMNFR21"})
                .drop_duplicates()
                .set_index("DTMNFR21"))

# Dados do Censo 2021

Fonte: https://mapas.ine.pt/download/filesGPG/2021Seccoes/

In [ ]:
!wget https://mapas.ine.pt/download/filesGPG/2021Seccoes/C2021_SECCOES_CONT.zip -P raw

In [3]:
seccoes_censo = gpd.read_file("raw/C2021_SECCOES_CONT.zip!C2021_SECCOES_CONT.gpkg") 

seccoes_censo = seccoes_censo.drop(
    columns=['DT21', 'DTMN21', 'DTMNFRSEC21', 'NUTS1_15','NUTS2_15','NUTS3_15','N_SS','SECNUM21']
)

In [4]:
seccoes_censo.is_valid.value_counts()

True    9929
Name: count, dtype: int64

## Group by freguesia

In [6]:
grouped_freguesia = seccoes_censo.dissolve(by="DTMNFR21", aggfunc='sum')

In [7]:
grouped_freguesia.is_valid.value_counts()

True    2882
Name: count, dtype: int64

## Adicionar nomes oficiais

In [9]:
grouped_freguesia_names = grouped_freguesia.join(codigos_df)

## Simplificar geografia

In [10]:
topo = tp.Topology(grouped_freguesia_names, prequantize=False)
simplified_data = topo.toposimplify(100).to_gdf()

simplified_data.is_valid.value_counts()

True     2860
False      22
Name: count, dtype: int64

In [11]:
simplified_data.geometry = simplified_data.geometry.make_valid()

In [12]:
simplified_data.to_parquet("../data/censo2021pt/censo2021_freguesia_toposimplify100.parquet", index=Falsse)

# Descrições de variáveis

Fonte: https://mapas.ine.pt/download/index2021Seccoes.phtml

In [35]:
!wget https://mapas.ine.pt/download/C2021_FSINTESE_VARIAVEIS_SECCOES.csv -P raw

--2024-09-24 01:03:20--  https://mapas.ine.pt/download/C2021_FSINTESE_VARIAVEIS_SECCOES.csv
Resolviendo mapas.ine.pt (mapas.ine.pt)... 193.192.10.123
Conectando con mapas.ine.pt (mapas.ine.pt)[193.192.10.123]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 16048 (16K) [text/plain]
Grabando a: «raw/C2021_FSINTESE_VARIAVEIS_SECCOES.csv»

C2021_FSINTESE_VARI 100%[===================>]  15.67K  --.-KB/s    en 0s      

2024-09-24 01:03:20 (14.9 GB/s) - «raw/C2021_FSINTESE_VARIAVEIS_SECCOES.csv» guardado [16048/16048]



In [33]:
variables = pd.read_csv("raw/C2021_FSINTESE_VARIAVEIS_SECCOES.csv", sep=";", skiprows=1).rename(columns={"Variável ": "column", "Descritivo": "description"})

missing_variables = pd.DataFrame([
    {"column": "Shape_Area", "description": "Área em m2"},
    {"column": "Shape_Length", "description": "Perímetro em m"}
])
missing_variables = pd.concat([variables, missing_variables], ignore_index=True)
missing_variables.to_csv("../data/censo2021pt/variables.csv", index=False)

# Embeddings

In [ ]:
all_embeddings = codigos_df.stack().reset_index(drop=True).drop_duplicates().dropna().sort_values().to_list()
len(all_embeddings)

In [5]:
gpt = OpenAIProvider()

In [15]:
requests = [generate_openai_embedding_request(i, text) for i, text in enumerate(all_embeddings, start=1) ]
request_id = gpt.send_batch_embedding(requests, "embedding_input.jsonl")

In [ ]:
batch = gpt.get_batch(request_id)

In [55]:
output = gpt.get_batch_result(batch.output_file_id, "embeddings_output.jsonl")

In [78]:
embedding_df = pd.DataFrame(zip(all_embeddings, map(lambda r:r["response"]["body"]["data"][0]["embedding"], output)), columns=["text", "values"])
embedding_df.to_parquet('../data/censo2021pt/embeddings.parquet.gz', index=False, compression="gzip")